In [3]:
from matplotlib import pyplot as plt
import numpy as np
import math
import meep as mp
import cmath

shape_size = 48

sx, sy, sz = 1, 1, 4
h = 1.25
dpml = 0.5
b_m, c_m = 1.4, 3.54
res = 15
echo = 1000
cell_size = mp.Vector3(sx,sy,sz)
fcen = 0.5
df = 0.2
theta = math.radians(0)
nfreq = 200

# k with correct length (plane of incidence: XZ) 
k = mp.Vector3(math.sin(theta),0,math.cos(theta)).scale(fcen)
def pw_amp(k, x0):
    def _pw_amp(x):
        return cmath.exp(1j * 2 * math.pi * k.dot(x + x0))
    return _pw_amp

def get_trans(vertices):
    geometry = [mp.Block(size = cell_size, material=mp.Medium(index=b_m)),
                mp.Prism(vertices, 
                         height=h, 
                         material=mp.Medium(index=c_m),
                         center=mp.Vector3()
                        )]
    pml_layers = [mp.PML(thickness=1, direction = mp.Z, side=mp.High),
                  mp.Absorber(thickness=1,direction = mp.Z, side=mp.Low)]
    src_pos = -(sz/2 - dpml - 0.5)
    src = [mp.Source(src = mp.GaussianSource(fcen, fwidth=df),
                     component = mp.Ey,
                     center = mp.Vector3(0,0,src_pos),
                     size = mp.Vector3(sx,sy,0),
                     amp_func=pw_amp(k,mp.Vector3(0,0,src_pos)))]
    sim = mp.Simulation(resolution=res,
                        cell_size=cell_size,
                        boundary_layers=pml_layers,
                        sources=src,
                        geometry=geometry,
                        k_point=k)
    freg = mp.FluxRegion(center=mp.Vector3(0,0,-src_pos),
                         size = mp.Vector3(sx,sy,0))
    trans = sim.add_flux(fcen, df, nfreq, freg)
    sim.run(until = echo)
    bend = mp.get_fluxes(trans)
    
    #get straight
    sim.reset_meep()
    geometry = [mp.Block(size = cell_size, material=mp.Medium(index=b_m))]
    pml_layers = [mp.PML(thickness= 1, direction = mp.Z, side=mp.High),
                  mp.Absorber(thickness=1,direction = mp.Z, side=mp.Low)]
    src = [mp.Source(src = mp.GaussianSource(fcen, fwidth=df),
                     component = mp.Ey,
                     center = mp.Vector3(0,0,src_pos),
                     size = mp.Vector3(sx,sy,0),
                     amp_func=pw_amp(k,mp.Vector3(0,0,src_pos)))]
    sim = mp.Simulation(resolution=res,
                        cell_size=cell_size,
                        boundary_layers=pml_layers,
                        sources=src,
                        geometry=geometry,
                        k_point=k)
    freg = mp.FluxRegion(center=mp.Vector3(0,0,-src_pos),
                         size = mp.Vector3(sx,sy,0))
    trans = sim.add_flux(fcen, df, nfreq, freg)
    sim.run(until = echo)
    straight = mp.get_fluxes(trans)
    flux_freqs = mp.get_flux_freqs(trans)
    
    c = 300
    p = 0.6
    Ts = []
    for i in range(nfreq):
        Ts = np.append(Ts, bend[i]/straight[i])
    return np.multiply(flux_freqs, c/p),Ts

In [4]:
def data_generator(batch):
    data = []
    # get shape from 'data/DATAX_sh.txt'
    coordinates = np.genfromtxt('data/DATA'+str(batch)+'_sh.txt')
    xc, yc = coordinates[:, 0], coordinates[:, 1]
    size = len(xc) // shape_size
    xc = np.reshape(xc, (size, shape_size))
    yc = np.reshape(yc, (size, shape_size))

    for i in range(size):
        # form shape
        vertices = [mp.Vector3(xc[i][0],yc[i][0])]
        for j in range(1, len(xc[i]) - 1):
            # eliminate duplicate point
            if xc[i][j] == xc[i][j - 1] and yc[i][j] == yc[i][j - 1]:
                continue
            vertices.append(mp.Vector3(xc[i][j], yc[i][j]))
        # calculate transmission
        _, Ts = get_trans(vertices)
        data.append(Ts)
    # save the spectrum to 'data/DATAX_sp.txt'
    np.savetxt('data/DATA'+str(batch)+'_sp.txt', data)
    x = np.genfromtxt('data/SP_xaxis.txt')
    y = np.np.genfromtxt('data/DATA'+str(batch)+'_sp.txt')
    for i in range(size):
        print('The '+str(i)+'th shape:')
        plt.axis('equal')
        plt.ylim(-0.5, 0.5)
        plt.xlim(-0.5, 0.5)
        plt.plot(xc[i], yc[i])
        plt.fill(xc[i], yc[i])
        plt.show()
        plt.ylim(0, 1.1)
        plt.plot(x, y[i])
        plt.show()

In [ ]:
start, end = 5, 6
for i in range(start, end):
    data_generator(i)

-----------
Initializing structure...
Meep: using complex fields.
Meep progress: 16.866666666666667/1000.0 = 1.7% done in 4.0s, 233.4s to go
Meep progress: 33.86666666666667/1000.0 = 3.4% done in 8.0s, 228.5s to go
Meep progress: 52.13333333333333/1000.0 = 5.2% done in 12.0s, 218.3s to go
Meep progress: 70.86666666666666/1000.0 = 7.1% done in 16.0s, 210.0s to go
Meep progress: 89.3/1000.0 = 8.9% done in 20.0s, 204.2s to go
Meep progress: 108.36666666666666/1000.0 = 10.8% done in 24.0s, 197.7s to go
Meep progress: 127.39999999999999/1000.0 = 12.7% done in 28.0s, 192.0s to go
Meep progress: 146.5/1000.0 = 14.7% done in 32.0s, 186.7s to go
Meep progress: 165.5/1000.0 = 16.6% done in 36.0s, 181.7s to go
Meep progress: 184.26666666666665/1000.0 = 18.4% done in 40.0s, 177.3s to go
Meep progress: 203.06666666666666/1000.0 = 20.3% done in 44.1s, 172.9s to go
Meep progress: 221.26666666666665/1000.0 = 22.1% done in 48.1s, 169.1s to go
Meep progress: 238.56666666666666/1000.0 = 23.9% done in 52.

Meep progress: 92.96666666666667/1000.0 = 9.3% done in 20.0s, 195.2s to go
Meep progress: 111.73333333333333/1000.0 = 11.2% done in 24.0s, 190.9s to go
Meep progress: 130.56666666666666/1000.0 = 13.1% done in 28.0s, 186.6s to go
Meep progress: 149.36666666666667/1000.0 = 14.9% done in 32.0s, 182.4s to go
Meep progress: 167.93333333333334/1000.0 = 16.8% done in 36.0s, 178.5s to go
Meep progress: 186.73333333333332/1000.0 = 18.7% done in 40.0s, 174.3s to go
Meep progress: 205.5/1000.0 = 20.6% done in 44.0s, 170.2s to go
Meep progress: 224.29999999999998/1000.0 = 22.4% done in 48.0s, 166.1s to go
Meep progress: 243.06666666666666/1000.0 = 24.3% done in 52.0s, 162.1s to go
Meep progress: 261.8666666666667/1000.0 = 26.2% done in 56.0s, 158.0s to go
Meep progress: 280.1666666666667/1000.0 = 28.0% done in 60.0s, 154.3s to go
Meep progress: 298.6333333333333/1000.0 = 29.9% done in 64.0s, 150.4s to go
Meep progress: 316.9/1000.0 = 31.7% done in 68.0s, 146.7s to go
Meep progress: 335.83333333333

Meep progress: 141.1/1000.0 = 14.1% done in 36.0s, 219.4s to go
Meep progress: 156.7/1000.0 = 15.7% done in 40.0s, 215.5s to go
Meep progress: 172.4/1000.0 = 17.2% done in 44.1s, 211.5s to go
Meep progress: 190.56666666666666/1000.0 = 19.1% done in 48.1s, 204.1s to go
Meep progress: 209.06666666666666/1000.0 = 20.9% done in 52.1s, 196.9s to go
Meep progress: 227.5/1000.0 = 22.8% done in 56.1s, 190.4s to go
